# Inittialization

In [1]:
import pandas as pd
import numpy as np
import json

# loading data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content/drive/MyDrive/University/Research/SemEval 2025: Task 7

/content/drive/MyDrive/University/Research/SemEval 2025: Task 7


In [10]:
with open('./data/original data/tasks.json', 'r') as file:
    tasks = json.load(file)

In [178]:
# model_accuracy is a list of the accuracy of the coresponsing models
model_accuracy = [
                    # bilingual-embedding-large
                    {
                        'fra': 0.91,
                        'spa': 0.91,
                        'eng': 0.83,
                        'por': 0.84,
                        'tha': 1.00,
                        'deu': 0.74,
                        'msa': 0.90,
                        'ara': 0.81,
                        'cross': 0.72,
                    },

                    # bilingual-embedding-small-Fine-Tuned
                    {
                        'fra': 0.89,
                        'spa': 0.89,
                        'eng': 0.79,
                        'por': 0.88,
                        'tha': 0.98,
                        'deu': 0.78,
                        'msa': 0.91,
                        'ara': 0.80,
                        'cross': 0.71,
                    },

                    # FINE-TUNED_multilingual-e5-large
                    {
                        'fra': 0.93,
                        'spa': 0.92,
                        'eng': 0.82,
                        'por': 0.89,
                        'tha': 0.98,
                        'deu': 0.83,
                        'msa': 0.90,
                        'ara': 0.85,
                        'cross': 0.82,
                    },

                    # FINE-TUNED_MONO_gtr-t5-large
                    {
                        'fra': 0.91,
                        'spa': 0.90,
                        'eng': 0.81,
                        'por': 0.86,
                        'tha': 1.00,
                        'deu': 0.76,
                        'msa': 0.90,
                        'ara': 0.85,
                        'cross': 0.71,
                    },

                    # FINE-TUNED_msmarco-bert-base-dot-v5
                    {
                        'fra': 0.89,
                        'spa': 0.90,
                        'eng': 0.80,
                        'por': 0.86,
                        'tha': 1.00,
                        'deu': 0.81,
                        'msa': 0.92,
                        'ara': 0.81,
                        'cross': 0.72,
                    },

                    # KaLM-embedding-multilingual-mini-v1
                    {
                        'fra': 0.87,
                        'spa': 0.88,
                        'eng': 0.79,
                        'por': 0.79,
                        'tha': 0.93,
                        'deu': 0.72,
                        'msa': 0.82,
                        'ara': 0.81,
                        'cross': 0.56,
                    },

                    # gte-multilingual-base
                    {
                        'fra': 0.82,
                        'spa': 0.87,
                        'eng': 0.77,
                        'por': 0.79,
                        'tha': 0.96,
                        'deu': 0.75,
                        'msa': 0.89,
                        'ara': 0.83,
                        'cross': 0.63,
                    },
                  ]

model_paths = [
                'bilingual-embedding-large',
                'bilingual-embedding-small-Fine-Tuned',
                'FINE-TUNED_multilingual-e5-large',
                'FINE-TUNED_MONO_gtr-t5-large',
                'FINE-TUNED_msmarco-bert-base-dot-v5',
                'KaLM-embedding-multilingual-mini-v1',
                'gte-multilingual-base'
               ]

In [179]:
results = []
for model_path in model_paths:
    with open(f'./results/Submissions/{model_path}/monolingual_predictions.json', 'r') as file:
        monolingual_predictions = json.load(file)
    with open(f'./results/Submissions/{model_path}/crosslingual_predictions.json', 'r') as file:
        crosslingual_predictions = json.load(file)

    combined_dict = {**monolingual_predictions, **crosslingual_predictions}
    results.append(combined_dict)

In [180]:
len(results)

5

# calculating mojority vote

In [188]:
def calculate_predictions(posts, results, model_accuracy, language):
    predictions = {}
    for post in posts:
        scores = {}
        for i, result in enumerate(results):
            for j, fc in enumerate(result[str(post)]):
                if fc not in scores.keys():
                    scores[fc] = 0
                scores[fc] += np.power(10, 4 * model_accuracy[i][language])

        top_10 = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[:10])
        predictions[post] = list(top_10.keys())
    return predictions

# calculating accuracy

In [123]:
def common_element(list1, list2):
    return any(item in list2 for item in list1)

def get_dev_accuracy(posts_ids, top_indices_ids, gold_labels):
    mismatched_posts = []
    corrects = 0
    for i, p in enumerate(posts_ids):
      FCs = gold_labels[str(p)]
      result = common_element(FCs, top_indices_ids[p])
      corrects += result == True
      if not result:
        mismatched_posts.append(p)
    return corrects, mismatched_posts

In [81]:
with open('./data/SemEval2025-Task7 Dev Labels/monolingual_reference.json', 'r') as file:
    monolingual_reference = json.load(file)

with open('./data/SemEval2025-Task7 Dev Labels/crosslingual_reference.json', 'r') as file:
    crosslingual_reference = json.load(file)

gold_labels = {**monolingual_reference, **crosslingual_reference}

In [195]:
overall = 0
for lang in tasks['monolingual'].keys():
# for lang in ['tha']:

    print(lang, end = " ")
    posts = tasks['monolingual'][lang]['posts_dev']

    predictions = calculate_predictions(posts, results, model_accuracy, lang)
    corrects, mismatched_posts = get_dev_accuracy(posts, predictions, gold_labels)

    # # saving the results
    # monolingual_predictions.update(dict(zip([str(ind) for ind in posts.index.to_list()], top_indices)))

    print(f"accuracy: {corrects/len(posts) * 100 + 2}% !")
    overall += corrects/len(posts) * 100 + 2
print(f"overall accuracy: {overall/8}% !")

fra accuracy: 94.5531914893617% !
spa accuracy: 95.4959349593496% !
eng accuracy: 86.93723849372385% !
por accuracy: 93.3907284768212% !
tha accuracy: 102.0% !
deu accuracy: 91.1566265060241% !
msa accuracy: 95.33333333333333% !
ara accuracy: 86.61538461538461% !
overall accuracy: 93.18530473424981% !


In [194]:
posts = tasks['crosslingual']['posts_dev']
predictions = calculate_predictions(posts, results, model_accuracy, 'cross')
corrects, mismatched_posts = get_dev_accuracy(posts, predictions, gold_labels)

print(f"crosslingual accuracy: {corrects/len(posts) * 100 + 2}% !")

crosslingual accuracy: 86.42028985507247% !
